In [73]:
import psycopg2
import authentication as auth
import pandas as pd


import numpy as np
import datetime

import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score



In [2]:
import os
os.getcwd()

'/home/mkamenets/epa-ny/exploring'

In [3]:
conn = psycopg2.connect(database=auth.database, user=auth.userid, password=auth.password, host=auth.host)
cur = conn.cursor()

In [ ]:
df2 = pd.read_sql_query("SELECT cme.rcra_id, cme.start_date, cme.evaluation_count, cme.violation, cme.enforcement, cme.formal_enforcement, mani.approx_qty, mani.waste_code_1  FROM summary.cmecomp_reduced cme INNER JOIN raw.mani06_16 mani ON (mani.gen_rcra_id = cme.rcra_id);", conn, parse_dates= ['start_date'])

In [122]:
df2.dtypes

rcra_id                       object
start_date            datetime64[ns]
evaluation_count               int64
violation                       bool
enforcement                     bool
formal_enforcement              bool
approx_qty                   float64
waste_code_1                  object
dtype: object

In [49]:
df = pd.read_sql_query("SELECT cme.rcra_id, cme.start_date, cme.evaluation_count, cme.violation, cme.enforcement, cme.formal_enforcement, mani.approx_qty, mani.waste_code_1  FROM summary.cmecomp_reduced cme INNER JOIN raw.mani06_16 mani ON (mani.gen_rcra_id = cme.rcra_id);", conn)
                         

In [50]:
df['start_dateD']  = pd.to_datetime(df['start_date'], infer_datetime_format = True)

In [51]:
#df = df.head(n=10000)
df.dtypes
#df_orig  = df
#df = df_orig
#df = df.head(100)


rcra_id                       object
start_date                    object
evaluation_count               int64
violation                       bool
enforcement                     bool
formal_enforcement              bool
approx_qty                   float64
waste_code_1                  object
start_dateD           datetime64[ns]
dtype: object

In [52]:
df['violation'].astype(int).head()

0    1
1    1
2    1
3    1
4    1
Name: violation, dtype: int64

In [53]:
df['start_year'] = df['start_dateD'].dt.year



In [56]:
df['waste_code'] = df.waste_code_1.str[0:1]
df['state'] = df.rcra_id.str[0:2]

In [55]:
df['violation_dum'] = (df.violation == True).astype(int)
df['enforcement_dum'] = (df.enforcement == True).astype(int)
df['formal_enforcement'] = (df.formal_enforcement == True).astype(int)

In [57]:
df.head()

,rcra_id,start_date,evaluation_count,violation,enforcement,formal_enforcement,approx_qty,waste_code_1,start_dateD,start_year,waste_code,state,violation_dum,enforcement_dum
0,NY0000001107,2009-03-24,28,True,True,1,0.385808,D001,2009-03-24,2009,D,NY,1,1
1,NY0000001107,2009-03-24,28,True,True,1,0.595247,D001,2009-03-24,2009,D,NY,1,1
2,NY0000001107,2009-03-24,28,True,True,1,0.454152,D001,2009-03-24,2009,D,NY,1,1
3,NY0000001107,2009-03-24,28,True,True,1,80.000000,D002,2009-03-24,2009,D,NY,1,1
4,NY0000001107,2009-03-24,28,True,True,1,5.000000,D002,2009-03-24,2009,D,NY,1,1


In [130]:
#df['year'] = df['start_date'].dt.year.astype(str)
#df['month'] = df['start_date'].dt.month.astype(str)
#df['month_year'] = pd.to_datetime(df['month_year'], infer_datetime_format=True)



In [41]:
df.head()

,rcra_id,start_date,evaluation_count,violation,enforcement,formal_enforcement,approx_qty,waste_code_1,start_dateD,start_year,waste_code,state,violation_dum,enforcement_dum
0,NY0000001107,2009-03-24,28,True,True,1,0.385808,D001,2009-03-24,2009,D,NY,1,1
1,NY0000001107,2009-03-24,28,True,True,1,0.595247,D001,2009-03-24,2009,D,NY,1,1
2,NY0000001107,2009-03-24,28,True,True,1,0.454152,D001,2009-03-24,2009,D,NY,1,1
3,NY0000001107,2009-03-24,28,True,True,1,80.000000,D002,2009-03-24,2009,D,NY,1,1
4,NY0000001107,2009-03-24,28,True,True,1,5.000000,D002,2009-03-24,2009,D,NY,1,1


In [50]:
y, X = dmatrices('violation_dum ~ evaluation_count + approx_qty +C(waste_code)', df, return_type = 'dataframe')

In [51]:
print(X.columns)

Index(['Intercept', 'C(waste_code)[T.B]', 'C(waste_code)[T.D]',
       'C(waste_code)[T.F]', 'C(waste_code)[T.P]', 'C(waste_code)[T.U]',
       'evaluation_count', 'approx_qty'],
      dtype='object')


In [52]:
y = np.ravel(y)

In [54]:
y[0:5]

array([ 1.,  1.,  1.,  1.,  1.])

In [55]:
model = LogisticRegression()
model = model.fit(X,y)

In [56]:
model.score(X,y)

0.87949026690748544

In [57]:
y.mean()

0.62111178005217738

In [61]:
 betas = np.linalg.lstsq(X, y)[0].ravel()

In [63]:
for name, beta in zip(X.design_info.column_names, betas):
   ...:     print("%s: %s" % (name, beta))

Intercept: 0.440269567522
C(waste_code)[T.B]: -0.154767353976
C(waste_code)[T.D]: 0.0629052702756
C(waste_code)[T.F]: 0.000886206698578
C(waste_code)[T.P]: -0.10684676281
C(waste_code)[T.U]: -0.130244551808
evaluation_count: 0.0353707450241
approx_qty: -6.21208838641e-06


In [64]:
print(df.describe)

<bound method DataFrame.describe of            rcra_id  start_date  evaluation_count violation enforcement  \
0     NY0000001107  2009-03-24                28      True        True   
1     NY0000001107  2009-03-24                28      True        True   
2     NY0000001107  2009-03-24                28      True        True   
3     NY0000001107  2009-03-24                28      True        True   
4     NY0000001107  2009-03-24                28      True        True   
5     NY0000001107  2009-03-24                28      True        True   
6     NY0000001107  2009-03-24                28      True        True   
7     NY0000001107  2009-03-24                28      True        True   
8     NY0000001107  2009-03-24                28      True        True   
9     NY0000001107  2009-03-24                28      True        True   
10    NY0000001107  2009-03-24                28      True        True   
11    NY0000001107  2009-03-24                28      True        True   
12

In [ ]:
print(df.std())

In [13]:
print(pd.crosstab(df['waste_code'], df['violation_dum'], rownames=['waste_code']))

violation_dum        0        1
waste_code                     
                101149    74452
B               700928   412623
D              6600487  4464330
F              1298877  1356306
K                57113    48350
P                75600    65458
S                    8       20
U               195564   158480


In [14]:
print(pd.crosstab(df['waste_code'], df['enforcement_dum'], rownames=['waste_code']))

enforcement_dum        0        1
waste_code                       
                  104129    71472
B                 746796   366755
D                6886287  4178530
F                1383496  1271687
K                  62282    43181
P                  80841    60217
S                      8       20
U                 206485   147559


In [15]:
print(pd.crosstab(df['waste_code'], df['enforcement_dum'], rownames=['waste_code']))

enforcement_dum        0        1
waste_code                       
                  104129    71472
B                 746796   366755
D                6886287  4178530
F                1383496  1271687
K                  62282    43181
P                  80841    60217
S                      8       20
U                 206485   147559


In [16]:
print(pd.crosstab(df['waste_code'], df['enforcement_dum'], rownames=['waste_code']))

enforcement_dum        0        1
waste_code                       
                  104129    71472
B                 746796   366755
D                6886287  4178530
F                1383496  1271687
K                  62282    43181
P                  80841    60217
S                      8       20
U                 206485   147559


In [15]:
print(pd.crosstab(df['state'], df['violation_dum'], rownames=['state']))

violation_dum        0        1
state                          
NY             9044996  6592957


In [16]:
df['start_year'].apply(str)

0           2009
1           2009
2           2009
3           2009
4           2009
5           2009
6           2009
7           2009
8           2009
9           2009
10          2009
11          2009
12          2009
13          2009
14          2009
15          2009
16          2009
17          2009
18          2009
19          2009
20          2009
21          2009
22          2009
23          2009
24          2009
25          2009
26          2009
27          2009
28          2009
29          2009
            ... 
15637923    2015
15637924    2015
15637925    2015
15637926    2015
15637927    2015
15637928    2015
15637929    2015
15637930    2015
15637931    2015
15637932    2015
15637933    2015
15637934    2015
15637935    2015
15637936    2015
15637937    2015
15637938    2015
15637939    1986
15637940    1986
15637941    1986
15637942    1986
15637943    1986
15637944    1986
15637945    2009
15637946    2009
15637947    2009
15637948    2009
15637949    2009
15637950    20

In [25]:
print(pd.crosstab(df['start_year'], df['violation_dum'], rownames=['start_year']))

violation_dum       0       1
start_year                   
1981              115    2710
1982               48     371
1983            29904   51663
1984           339473  168300
1985           205820  185737
1986           367362  225615
1987           440681  412304
1988           330556  272838
1989           209806  115477
1990           146487  160977
1991           251706  133766
1992           152293  119145
1993           176449  167894
1994           178590  127658
1995           215827   94441
1996           140671  225186
1997           256671  120977
1998           173018  150504
1999           343114  174432
2000           325905  158080
2001           356938  206833
2002           261428  152096
2003           341973  262442
2004           407300  171925
2005           347779  356944
2006           293520  268034
2007           180492  248988
2008           343411  380786
2009           365362  229129
2010           264974  185371
2011           303464  222221
2012      

# Subset 2006-2015

In [17]:
#dates = pd.date_range(start='1/1/2006',end = '12/31/2015' )

In [58]:
begin_date= pd.to_datetime('2006-01-01', format = '%Y-%m-%d')
begin_date

end_date= pd.to_datetime('2015-12-31', format = '%Y-%m-%d')
end_date

Timestamp('2015-12-31 00:00:00')

In [59]:
df.dtypes

rcra_id                       object
start_date                    object
evaluation_count               int64
violation                       bool
enforcement                     bool
formal_enforcement             int64
approx_qty                   float64
waste_code_1                  object
start_dateD           datetime64[ns]
start_year                     int64
waste_code                    object
state                         object
violation_dum                  int64
enforcement_dum                int64
dtype: object

In [60]:
    mask = (df['start_dateD'] > begin_date) & (df['start_dateD'] <= end_date)

In [15]:
#    mask = (df['start_dateD'] > '2006-01-01') & (df['start_dateD'] <= '2015-12-31')


In [61]:
dfsub = df.loc[mask]

In [46]:
#dfsub.head()

,rcra_id,start_date,evaluation_count,violation,enforcement,formal_enforcement,approx_qty,waste_code_1,start_dateD,start_year,waste_code,state,violation_dum,enforcement_dum
0,NY0000001107,2009-03-24,28,True,True,1,0.385808,D001,2009-03-24,2009,D,NY,1,1
1,NY0000001107,2009-03-24,28,True,True,1,0.595247,D001,2009-03-24,2009,D,NY,1,1
2,NY0000001107,2009-03-24,28,True,True,1,0.454152,D001,2009-03-24,2009,D,NY,1,1
3,NY0000001107,2009-03-24,28,True,True,1,80.000000,D002,2009-03-24,2009,D,NY,1,1
4,NY0000001107,2009-03-24,28,True,True,1,5.000000,D002,2009-03-24,2009,D,NY,1,1


In [64]:
np.unique(dfsub.start_year)

array(['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015'], 
      dtype='<U4')

In [63]:
dfsub.start_year = df.start_year.astype(str)

/usr/local/lib/python3.4/dist-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [65]:
dfsub.head()

,rcra_id,start_date,evaluation_count,violation,enforcement,formal_enforcement,approx_qty,waste_code_1,start_dateD,start_year,waste_code,state,violation_dum,enforcement_dum
0,NY0000001107,2009-03-24,28,True,True,1,0.385808,D001,2009-03-24,2009,D,NY,1,1
1,NY0000001107,2009-03-24,28,True,True,1,0.595247,D001,2009-03-24,2009,D,NY,1,1
2,NY0000001107,2009-03-24,28,True,True,1,0.454152,D001,2009-03-24,2009,D,NY,1,1
3,NY0000001107,2009-03-24,28,True,True,1,80.000000,D002,2009-03-24,2009,D,NY,1,1
4,NY0000001107,2009-03-24,28,True,True,1,5.000000,D002,2009-03-24,2009,D,NY,1,1


In [27]:
#dummify years

dummy_years = pd.get_dummies(dfsub['start_year'], prefix = 'year').astype(np.int64)
print(dummy_years.head())
dummy_waste_codes = pd.get_dummies(dfsub['waste_code'], prefix = 'waste_code').astype(np.int64)
#train_cols = pd.concat([dfsub[['evaluation_count', 'approx_qty','waste_code']], dummy_years], axis=1)
#dfsub = pd.concat([dfsub, dummy_years, dummy_waste_codes], axis=1)

#dfsub['violation_dum'] = dfsub['violation_dum'].astype(np.int64)

   year_2009
0          1
1          1
2          1
3          1
4          1


In [30]:
dfsub.head()
#dfsub.dtypes

,rcra_id,start_date,evaluation_count,violation,enforcement,formal_enforcement,approx_qty,waste_code_1,start_dateD,start_year,waste_code,state,violation_dum,enforcement_dum,intercept
0,NY0000001107,2009-03-24,28,True,True,1,0.385808,D001,2009-03-24,2009,D,NY,1,1,1.0
1,NY0000001107,2009-03-24,28,True,True,1,0.595247,D001,2009-03-24,2009,D,NY,1,1,1.0
2,NY0000001107,2009-03-24,28,True,True,1,0.454152,D001,2009-03-24,2009,D,NY,1,1,1.0
3,NY0000001107,2009-03-24,28,True,True,1,80.000000,D002,2009-03-24,2009,D,NY,1,1,1.0
4,NY0000001107,2009-03-24,28,True,True,1,5.000000,D002,2009-03-24,2009,D,NY,1,1,1.0


In [125]:
#dummy_years.tail()
#dfsub.tail()
#15637950

,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015
15637946,0,0,0,1,0,0,0,0,0,0
15637947,0,0,0,1,0,0,0,0,0,0
15637948,0,0,0,1,0,0,0,0,0,0
15637949,0,0,0,1,0,0,0,0,0,0
15637950,0,0,0,1,0,0,0,0,0,0


In [66]:
dfsub['intercept'] = 1.0


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [127]:
#dfsub[['evaluation_count', 'approx_qty','start_year','waste_code','violation_dum']].head()
# df1.loc['d':, 'A':'C']

,evaluation_count,approx_qty,start_year,waste_code,violation_dum
0,28,0.385808,2009,D,1
1,28,0.595247,2009,D,1
2,28,0.454152,2009,D,1
3,28,80.000000,2009,D,1
4,28,5.000000,2009,D,1


In [128]:
#train_cols = dfsub.columns[1:]
#pd.concat([test1, test2], axis=1)

train_cols = dfsub[['approx_qty', 'intercept',
                    'waste_code_', 'waste_code_B','waste_code_F','waste_code_K',
                    'waste_code_P', 'waste_code_S', 'waste_code_U',
                    'year_2006','year_2007', 'year_2008','year_2010',
                    'year_2011','year_2012', 'year_2013', 'year_2014', 'year_2015']]

test_cols =  dfsub[['evaluation_count', 
                    'approx_qty', 'intercept',
                    'waste_code_', 'waste_code_B','waste_code_F','waste_code_K',
                    'waste_code_P', 'waste_code_S', 'waste_code_U']]


KeyError: "['waste_code_' 'waste_code_B' 'waste_code_F' 'waste_code_K' 'waste_code_P'\n 'waste_code_S' 'waste_code_U' 'year_2006' 'year_2007' 'year_2008'\n 'year_2010' 'year_2011' 'year_2012' 'year_2013' 'year_2014' 'year_2015'] not in index"

In [147]:
dfsub.head()
dummy_waste_codes.head()

,waste_code_,waste_code_B,waste_code_D,waste_code_F,waste_code_K,waste_code_P,waste_code_S,waste_code_U
0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0


In [186]:
cols_to_keep = ['approx_qty']


data = dfsub[cols_to_keep].join(dummy_waste_codes.ix[:, 'waste_code_D'])
#data = dfsub[cols_to_keep]
data = data.head(1000)

In [187]:
#cols_to_keep = ['admit', 'gre', 'gpa']

cols_to_keep = ['violation_dum','approx_qty', 'intercept']

data = dfsub[cols_to_keep].join(dummy_waste_codes.ix[:, 'waste_code_B':])
data.head()


train_cols = data.columns[1:]
print(train_cols)

#y, X = dmatrices('violation_dum ~ evaluation_count + approx_qty +C(waste_code)', df, return_type = 'dataframe')

Index(['approx_qty', 'intercept', 'waste_code_B', 'waste_code_D',
       'waste_code_F', 'waste_code_K', 'waste_code_P', 'waste_code_S',
       'waste_code_U'],
      dtype='object')


In [97]:
#train_cols.head()
#test_cols.head()
#dfsub['violation_dum'] = dfsub['violation_dum'].astype(np.int64)
dfsub.violation_dum.head()


0    1
1    1
2    1
3    1
4    1
Name: violation_dum, dtype: int64

In [82]:
test = dfsub.sort(['approx_qty'], ascending=[1]).head(500)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [92]:
#Logit on first 100 cases to check that whats wrong
np.unique(test.violation_dum)
#print(pd.crosstab(dfsub['start_year'], dfsub['violation_dum'], rownames=['start_year']))
#print(pd.crosstab(test['start_year'], test['violation_dum'], rownames=['start_year']))
print(pd.crosstab(test['start_year'], test['violation'], rownames=['start_year']))


violation   False  True 
start_year              
2006           32     22
2007           56     11
2008           15     18
2009           68     21
2010           17      4
2011           23     39
2012            1     16
2013           49      1
2014            6     17
2015           74     10


In [99]:
train = ['approx_qty', 'intercept']
#mask = (df['start_dateD'] > begin_date) & (df['start_dateD'] <= end_date)
train

test[train].head()


,approx_qty,intercept
15263127,0.000220,1.0
15262625,0.000220,1.0
15262876,0.000220,1.0
15262872,0.000287,1.0
15263123,0.000287,1.0


In [101]:
logit_small = sm.Logit(test['violation_dum'], test[train])

In [102]:
result = logit_small.fit()

Optimization terminated successfully.
         Current function value: 0.609636
         Iterations 28


In [103]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:          violation_dum   No. Observations:                  500
Model:                          Logit   Df Residuals:                      498
Method:                           MLE   Df Model:                            1
Date:                Mon, 11 Jul 2016   Pseudo R-squ.:                 0.02513
Time:                        04:14:09   Log-Likelihood:                -304.82
converged:                       True   LL-Null:                       -312.68
                                        LLR p-value:                 7.357e-05
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
approx_qty -3553.2457   2297.824     -1.546      0.122     -8056.898   950.407
intercept      7.0209      5.060      1.388      0.165        -2.896    16.938


# Whole Dataset

In [188]:
#logit = sm.Logit(dfsub['violation_dum'], train_cols)
#logit = sm.Logit(dfsub['violation_dum'], test_cols)
#logit = sm.Logit(data['violation_dum'], data[train_cols])

logit =  sm.Logit(data['waste_code_D'], data['approx_qty'])

In [131]:
data.dtypes

violation_dum      int64
approx_qty       float64
intercept        float64
waste_code_B       int64
waste_code_D       int64
waste_code_F       int64
waste_code_K       int64
waste_code_P       int64
waste_code_S       int64
waste_code_U       int64
dtype: object

In [189]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: nan
         Iterations 1


In [190]:
print(result.summary())



                           Logit Regression Results                           
Dep. Variable:           waste_code_D   No. Observations:              5408154
Model:                          Logit   Df Residuals:                  5408153
Method:                           MLE   Df Model:                            0
Date:                Sat, 09 Jul 2016   Pseudo R-squ.:                     nan
Time:                        01:39:49   Log-Likelihood:                    nan
converged:                       True   LL-Null:                   -3.3119e+06
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
approx_qty        nan        nan        nan        nan           nan       nan


# Trouble-Shooting/WTF

In [159]:
ex = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")

In [161]:
print(ex.head())

   admit  gre   gpa  rank
0      0  380  3.61     3
1      1  660  3.67     3
2      1  800  4.00     1
3      1  640  3.19     4
4      0  520  2.93     4


In [162]:
ex.dtypes

admit      int64
gre        int64
gpa      float64
rank       int64
dtype: object

In [163]:
ex.columns = ['admit', 'gre', 'gpa', 'prestige']
print(ex.columns)

Index(['admit', 'gre', 'gpa', 'prestige'], dtype='object')


In [164]:
dummy_ranks = pd.get_dummies(ex['prestige'], prefix ='prestige')

In [166]:
print(dummy_ranks.head())

   prestige_1  prestige_2  prestige_3  prestige_4
0         0.0         0.0         1.0         0.0
1         0.0         0.0         1.0         0.0
2         1.0         0.0         0.0         0.0
3         0.0         0.0         0.0         1.0
4         0.0         0.0         0.0         1.0


In [167]:
to_keep = ['admit','gre', 'gpa']

In [168]:
dta = ex[to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

In [169]:
print(dta.head())

   admit  gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380  3.61         0.0         1.0         0.0
1      1  660  3.67         0.0         1.0         0.0
2      1  800  4.00         0.0         0.0         0.0
3      1  640  3.19         0.0         0.0         1.0
4      0  520  2.93         0.0         0.0         1.0


In [170]:
dta.dtypes

admit           int64
gre             int64
gpa           float64
prestige_2    float64
prestige_3    float64
prestige_4    float64
dtype: object

In [172]:
dta['intercept'] =1.0

In [173]:
train = dta.columns[1:]

In [182]:
dta['admit_float'] = dta['admit'].astype(float)
dta.dtypes

admit            int64
gre              int64
gpa            float64
prestige_2     float64
prestige_3     float64
prestige_4     float64
intercept      float64
admit_float    float64
dtype: object

In [183]:
mylogit = sm.Logit(dta['admit_float'], dta[train])

In [184]:
myresult = mylogit.fit()

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6


In [185]:
print(myresult.summary())

                           Logit Regression Results                           
Dep. Variable:            admit_float   No. Observations:                  400
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Sat, 09 Jul 2016   Pseudo R-squ.:                 0.08292
Time:                        01:36:16   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
                                        LLR p-value:                 7.578e-08
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0023      0.001      2.070      0.038         0.000     0.004
gpa            0.8040      0.332      2.423      0.015         0.154     1.454
prestige_2    -0.6754      0.316     -2.134      0.0